In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1048037,2021-03-02,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1048038,2021-03-03,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1048039,2021-03-04,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1048040,2021-03-05,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
28383,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
28385,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
28387,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
28389,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
28391,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
648726,2021-03-02,Arkansas,Arkansas,5001,1996,33.00,5000,Arkansas,AR,Arkansas,State,3017804
648728,2021-03-03,Arkansas,Arkansas,5001,1997,33.00,5000,Arkansas,AR,Arkansas,State,3017804
648730,2021-03-04,Arkansas,Arkansas,5001,1999,33.00,5000,Arkansas,AR,Arkansas,State,3017804
648732,2021-03-05,Arkansas,Arkansas,5001,2002,33.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1048037,2021-03-02,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1048038,2021-03-03,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1048039,2021-03-04,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1048040,2021-03-05,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-03-06') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
410,2021-03-06,Snohomish,Washington,31299,546.00,53061,WA,County,822083,3807.28,66.42
818,2021-03-06,Cook,Illinois,478498,10002.00,17031,IL,County,5150233,9290.80,194.20
1225,2021-03-06,Orange,California,262550,4173.00,6059,CA,County,3175692,8267.49,131.40
1631,2021-03-06,Maricopa,Arizona,515211,9314.00,4013,AZ,County,4485414,11486.36,207.65
2037,2021-03-06,Los Angeles,California,1202513,22008.00,6037,CA,County,10039107,11978.29,219.22
...,...,...,...,...,...,...,...,...,...,...,...
1047585,2021-03-06,Wheeler,Oregon,22,1.00,41069,OR,County,1332,1651.65,75.08
1047730,2021-03-06,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00
1047844,2021-03-06,Esmeralda,Nevada,37,0.00,32009,NV,County,873,4238.26,0.00
1047954,2021-03-06,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
410,2021-03-06,Snohomish,Washington,31299,546.00,53061,WA,County,822083,3807.28,66.42,66.42,3807.28
818,2021-03-06,Cook,Illinois,478498,10002.00,17031,IL,County,5150233,9290.80,194.20,194.20,9290.80
1225,2021-03-06,Orange,California,262550,4173.00,6059,CA,County,3175692,8267.49,131.40,131.40,8267.49
1631,2021-03-06,Maricopa,Arizona,515211,9314.00,4013,AZ,County,4485414,11486.36,207.65,207.65,11486.36
2037,2021-03-06,Los Angeles,California,1202513,22008.00,6037,CA,County,10039107,11978.29,219.22,219.22,11978.29
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1047585,2021-03-06,Wheeler,Oregon,22,1.00,41069,OR,County,1332,1651.65,75.08,75.08,1651.65
1047730,2021-03-06,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00,0.00,4044.12
1047844,2021-03-06,Esmeralda,Nevada,37,0.00,32009,NV,County,873,4238.26,0.00,0.00,4238.26
1047954,2021-03-06,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00,0.00,591.72


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
737999,2021-03-06,Clarke,Alabama,3445,53.00,1025,AL,County,23622,14583.86,224.37,224.37,14583.86,1
678229,2021-03-06,Hale,Alabama,2125,69.00,1065,AL,County,14651,14504.13,470.96,470.96,14504.13,2
531081,2021-03-06,Lowndes,Alabama,1331,51.00,1085,AL,County,9726,13684.97,524.37,524.37,13684.97,3
471056,2021-03-06,Etowah,Alabama,13413,327.00,1055,AL,County,102268,13115.54,319.75,319.75,13115.54,4
531427,2021-03-06,Marengo,Alabama,2446,57.00,1091,AL,County,18863,12967.18,302.18,302.18,12967.18,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
647361,2021-03-06,Converse,Wyoming,970,16.00,56009,WY,County,13822,7017.80,115.76,115.76,7017.80,19
647705,2021-03-06,Sublette,Wyoming,679,7.00,56035,WY,County,9831,6906.72,71.20,71.20,6906.72,20
894372,2021-03-06,Niobrara,Wyoming,150,2.00,56027,WY,County,2356,6366.72,84.89,84.89,6366.72,21
804106,2021-03-06,Lincoln,Wyoming,1248,11.00,56023,WY,County,19830,6293.49,55.47,55.47,6293.49,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
531081,2021-03-06,Lowndes,Alabama,1331,51.00,1085,AL,County,9726,13684.97,524.37,524.37,13684.97,3,1
678229,2021-03-06,Hale,Alabama,2125,69.00,1065,AL,County,14651,14504.13,470.96,470.96,14504.13,2,2
226355,2021-03-06,Walker,Alabama,6600,260.00,1127,AL,County,63521,10390.26,409.31,409.31,10390.26,31,3
469668,2021-03-06,Clay,Alabama,1462,54.00,1027,AL,County,13235,11046.47,408.01,408.01,11046.47,20,4
530389,2021-03-06,Crenshaw,Alabama,1467,55.00,1041,AL,County,13772,10652.05,399.36,399.36,10652.05,30,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
571217,2021-03-06,Hot Springs,Wyoming,361,3.00,56017,WY,County,4413,8180.38,67.98,67.98,8180.38,17,19
784164,2021-03-06,Uinta,Wyoming,2070,12.00,56041,WY,County,20226,10234.35,59.33,59.33,10234.35,4,20
804106,2021-03-06,Lincoln,Wyoming,1248,11.00,56023,WY,County,19830,6293.49,55.47,55.47,6293.49,22,21
225296,2021-03-06,Teton,Wyoming,3375,9.00,56039,WY,County,23464,14383.74,38.36,38.36,14383.74,1,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,66.42,3807.28,14,21
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,66.42,3807.28,14,21
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,66.42,3807.28,14,21
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,66.42,3807.28,14,21
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,66.42,3807.28,14,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1045566,2021-03-02,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3
1045567,2021-03-03,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3
1045568,2021-03-04,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3
1045569,2021-03-05,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
735189,2020-04-01,Clarke,Alabama,2,0.00,1025,AL,County,23622,8.47,0.00,224.37,14583.86,39,1,2.00
735190,2020-04-02,Clarke,Alabama,2,0.00,1025,AL,County,23622,8.47,0.00,224.37,14583.86,39,1,0.00
735191,2020-04-03,Clarke,Alabama,3,0.00,1025,AL,County,23622,12.70,0.00,224.37,14583.86,39,1,1.00
735192,2020-04-04,Clarke,Alabama,7,0.00,1025,AL,County,23622,29.63,0.00,224.37,14583.86,39,1,4.00
735193,2020-04-05,Clarke,Alabama,9,0.00,1025,AL,County,23622,38.10,0.00,224.37,14583.86,39,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
817470,2021-03-02,Crook,Wyoming,418,11.00,56011,WY,County,7584,5511.60,145.04,145.04,5511.60,7,23,1.00
817471,2021-03-03,Crook,Wyoming,418,11.00,56011,WY,County,7584,5511.60,145.04,145.04,5511.60,7,23,0.00
817472,2021-03-04,Crook,Wyoming,418,11.00,56011,WY,County,7584,5511.60,145.04,145.04,5511.60,7,23,0.00
817473,2021-03-05,Crook,Wyoming,418,11.00,56011,WY,County,7584,5511.60,145.04,145.04,5511.60,7,23,0.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
528953,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,524.37,13684.97,1,3,1.00,0.00
528954,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,524.37,13684.97,1,3,0.00,0.00
528955,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,524.37,13684.97,1,3,0.00,0.00
528956,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,524.37,13684.97,1,3,0.00,0.00
528957,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,524.37,13684.97,1,3,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
527218,2021-03-02,Albany,Wyoming,3911,11.00,56001,WY,County,38880,10059.16,28.29,28.29,10064.30,23,6,2.00,0.00
527219,2021-03-03,Albany,Wyoming,3911,11.00,56001,WY,County,38880,10059.16,28.29,28.29,10064.30,23,6,0.00,0.00
527220,2021-03-04,Albany,Wyoming,3912,11.00,56001,WY,County,38880,10061.73,28.29,28.29,10064.30,23,6,1.00,0.00
527221,2021-03-05,Albany,Wyoming,3913,11.00,56001,WY,County,38880,10064.30,28.29,28.29,10064.30,23,6,1.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-03-06') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
270095,2021-03-06,Imperial,California,26970,638.00,6025,CA,County,181215,14882.87,352.07,352.07,14882.87,1,2,24.00,0.00
2037,2021-03-06,Los Angeles,California,1202513,22008.00,6037,CA,County,10039107,11978.29,219.22,219.22,11978.29,2,5,1749.00,98.00
472396,2021-03-06,Inyo,California,1309,37.00,6027,CA,County,18039,7256.50,205.11,205.11,7256.50,3,26,9.00,0.00
60341,2021-03-06,Stanislaus,California,56702,957.00,6099,CA,County,550660,10297.10,173.79,173.79,10297.10,4,10,100.00,0.00
74778,2021-03-06,Tulare,California,48203,770.00,6107,CA,County,466195,10339.66,165.17,165.17,10339.66,5,9,0.00,0.00
28017,2021-03-06,Riverside,California,290904,3892.00,6065,CA,County,2470546,11774.89,157.54,157.54,11774.89,6,6,0.00,0.00
48035,2021-03-06,San Joaquin,California,67284,1182.00,6077,CA,County,762148,8828.21,155.09,155.09,8828.21,7,18,0.00,0.00
27287,2021-03-06,Fresno,California,96184,1488.00,6019,CA,County,999101,9627.05,148.93,148.93,9627.05,8,14,161.00,22.00
428217,2021-03-06,Merced,California,29376,413.00,6047,CA,County,277680,10579.08,148.73,148.73,10579.08,9,8,0.00,0.00
572540,2021-03-06,Kings,California,22175,227.00,6031,CA,County,152940,14499.15,148.42,148.42,14499.15,10,3,25.00,0.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
995740,2021-03-06,Lassen,California,5623,24.00,6035,CA,County,30573,18392.05,78.50,78.50,18392.05,30,1,0.00,5.00
270095,2021-03-06,Imperial,California,26970,638.00,6025,CA,County,181215,14882.87,352.07,352.07,14882.87,1,2,24.00,0.00
572540,2021-03-06,Kings,California,22175,227.00,6031,CA,County,152940,14499.15,148.42,148.42,14499.15,10,3,25.00,0.00
134707,2021-03-06,San Bernardino,California,287782,3137.00,6071,CA,County,2180085,13200.49,143.89,143.89,13200.49,11,4,269.00,21.00
2037,2021-03-06,Los Angeles,California,1202513,22008.00,6037,CA,County,10039107,11978.29,219.22,219.22,11978.29,2,5,1749.00,98.00
28017,2021-03-06,Riverside,California,290904,3892.00,6065,CA,County,2470546,11774.89,157.54,157.54,11774.89,6,6,0.00,0.00
169281,2021-03-06,Kern,California,104181,942.00,6029,CA,County,900202,11573.07,104.64,104.64,11573.07,16,7,124.00,5.00
428217,2021-03-06,Merced,California,29376,413.00,6047,CA,County,277680,10579.08,148.73,148.73,10579.08,9,8,0.00,0.00
74778,2021-03-06,Tulare,California,48203,770.00,6107,CA,County,466195,10339.66,165.17,165.17,10339.66,5,9,0.00,0.00
60341,2021-03-06,Stanislaus,California,56702,957.00,6099,CA,County,550660,10297.10,173.79,173.79,10297.10,4,10,100.00,0.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
270095,2021-03-06,Imperial,California,26970,638.00,6025,CA,County,181215,14882.87,352.07,352.07,14882.87,1,2,24.00,0.00
2037,2021-03-06,Los Angeles,California,1202513,22008.00,6037,CA,County,10039107,11978.29,219.22,219.22,11978.29,2,5,1749.00,98.00
472396,2021-03-06,Inyo,California,1309,37.00,6027,CA,County,18039,7256.50,205.11,205.11,7256.50,3,26,9.00,0.00
60341,2021-03-06,Stanislaus,California,56702,957.00,6099,CA,County,550660,10297.10,173.79,173.79,10297.10,4,10,100.00,0.00
74778,2021-03-06,Tulare,California,48203,770.00,6107,CA,County,466195,10339.66,165.17,165.17,10339.66,5,9,0.00,0.00
28017,2021-03-06,Riverside,California,290904,3892.00,6065,CA,County,2470546,11774.89,157.54,157.54,11774.89,6,6,0.00,0.00
48035,2021-03-06,San Joaquin,California,67284,1182.00,6077,CA,County,762148,8828.21,155.09,155.09,8828.21,7,18,0.00,0.00
27287,2021-03-06,Fresno,California,96184,1488.00,6019,CA,County,999101,9627.05,148.93,148.93,9627.05,8,14,161.00,22.00
428217,2021-03-06,Merced,California,29376,413.00,6047,CA,County,277680,10579.08,148.73,148.73,10579.08,9,8,0.00,0.00
572540,2021-03-06,Kings,California,22175,227.00,6031,CA,County,152940,14499.15,148.42,148.42,14499.15,10,3,25.00,0.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_case_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,352.07,14882.87,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,352.07,14882.87,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,352.07,14882.87,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,352.07,14882.87,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,352.07,14882.87,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
3532,03/02/21,Lassen,5601,19.00,18320.09,62.15,78.50,18392.05,30,1,0.00,0.00
3533,03/03/21,Lassen,5623,19.00,18392.05,62.15,78.50,18392.05,30,1,22.00,0.00
3534,03/04/21,Lassen,5623,19.00,18392.05,62.15,78.50,18392.05,30,1,0.00,0.00
3535,03/05/21,Lassen,5623,19.00,18392.05,62.15,78.50,18392.05,30,1,0.00,0.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_death_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,352.07,14882.87,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,352.07,14882.87,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,352.07,14882.87,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,352.07,14882.87,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,352.07,14882.87,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
3606,03/02/21,Kings,22091,220.00,14444.23,143.85,148.42,14499.15,10,3,9.00,0.00
3607,03/03/21,Kings,22091,220.00,14444.23,143.85,148.42,14499.15,10,3,0.00,0.00
3608,03/04/21,Kings,22119,227.00,14462.53,148.42,148.42,14499.15,10,3,28.00,7.00
3609,03/05/21,Kings,22150,227.00,14482.80,148.42,148.42,14499.15,10,3,31.00,0.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)